In [2]:
import os
import sys
sys.path.append('../train')

import numpy as np
import torch


import model_bank
from read_data import *
from my_dataloader import *

/home/dell/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
component_dir = '../../sdsvc_exp/backend_exp/sdsvc_fusion'

In [6]:
if not os.path.isdir(component_dir):
    os.makedirs(component_dir)

In [9]:
sdsvc_trial_list = '/Lun0/zhiyong/sdsvc_small_100_dataset/sdsvc_trial_list.csv'
sdsvc_trial_keys = '/Lun0/zhiyong/sdsvc_small_100_dataset/sdsvc_trial_keys.csv'

# model 1

In [10]:
AUX_DATA_NAME_1 = 'AUX_DATA_1'

In [11]:
model_path = '/Lun2/rzz/kaldi-master/egs/zhiyong/sre19/exp/Standard_ETDNN_plus/ckpt/min_eer.model'
model_id = 'Standard_ETDNN_plus'
model_metric = 'AM_normfree_softmax_anneal_ce_head'

In [13]:
# train_list = '/Lun0/zhiyong/dataset/vox1_kaldi_test/vox1_kaldi_test.csv'
train_list = sdsvc_trial_list
test_list = {}
model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 10249, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

torch.backends.cudnn.benchmark = False
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device("cuda:0")

# train_data = CSVDataSet(train_list)
# train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=True)
train_data = PickleDataSet(train_list)
train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
batch_sampler=None, num_workers=8, collate_fn=None,\
pin_memory=False, drop_last=False, timeout=0,\
worker_init_fn=None, multiprocessing_context=None)

model = model_bank.get_model(model_id, model_metric, model_settings, _)
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint['model'], strict=True)
model = model.to(device)
model.eval()

for count, (batch_x, batch_y) in enumerate(train_dataloader):
    batch_x = batch_x.to(device)
    label = batch_y[0]

    batch_y = torch.tensor([0]).to(device)
    
    with torch.no_grad():
        _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

    emb = emb.squeeze().data.cpu().numpy()
    
    if label not in test_list.keys():
        test_list[label] = emb[None, :]
    else:
        print('repeat eer:', label)
        break
#         test_list[label] = np.append(test_list[label], emb[None, :], axis=0)
    
    if (count+1) % 500 == 0:
        print(count+1)

del model, batch_x, batch_y
torch.cuda.empty_cache()

Using model bank
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000


In [14]:
count+1

14490

In [15]:
len(test_list)

14490

In [16]:
import pickle
out = component_dir+'/'+AUX_DATA_NAME_1
with open(out, 'wb') as handle:
    pickle.dump(test_list, handle)

# model 2

In [17]:
AUX_DATA_NAME_2 = 'AUX_DATA_2'

In [18]:
model_path = '/Lun2/rzz/kaldi-master/egs/zhiyong/sre19/exp/Standard_ETDNN/ckpt/min_eer.model'
model_id = 'Standard_ETDNN'
model_metric = 'AM_normfree_softmax_anneal_ce_head'

In [19]:
# train_list = '/Lun0/zhiyong/dataset/vox1_kaldi_test/vox1_kaldi_test.csv'
train_list = sdsvc_trial_list
test_list = {}
model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 10249, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

torch.backends.cudnn.benchmark = False
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device("cuda:0")

# train_data = CSVDataSet(train_list)
# train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=True)
train_data = PickleDataSet(train_list)
train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
batch_sampler=None, num_workers=8, collate_fn=None,\
pin_memory=False, drop_last=False, timeout=0,\
worker_init_fn=None, multiprocessing_context=None)

model = model_bank.get_model(model_id, model_metric, model_settings, _)
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint['model'], strict=True)
model = model.to(device)
model.eval()

for count, (batch_x, batch_y) in enumerate(train_dataloader):
    batch_x = batch_x.to(device)
    label = batch_y[0]

    batch_y = torch.tensor([0]).to(device)
    
    with torch.no_grad():
        _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

    emb = emb.squeeze().data.cpu().numpy()
    
    if label not in test_list.keys():
        test_list[label] = emb[None, :]
    else:
        print('repeat eer:', label)
        break
#         test_list[label] = np.append(test_list[label], emb[None, :], axis=0)
    
    if (count+1) % 500 == 0:
        print(count+1)

del model, batch_x, batch_y
torch.cuda.empty_cache()

Using model bank
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000


In [20]:
count+1

14490

In [21]:
len(test_list)

14490

In [22]:
import pickle
out = component_dir+'/'+AUX_DATA_NAME_2
with open(out, 'wb') as handle:
    pickle.dump(test_list, handle)

# model 3

In [23]:
AUX_DATA_NAME_3 = 'AUX_DATA_3'

In [24]:
model_path = '/Lun2/rzz/kaldi-master/egs/zhiyong/sre19/exp/Xvector_1L_fullevad_sparse/ckpt/min_eer.model'
model_id = 'Xvector_SAP_1L'
model_metric = 'AM_normfree_softmax_anneal_ce_head'

In [25]:
# train_list = '/Lun0/zhiyong/dataset/vox1_kaldi_test/vox1_kaldi_test.csv'
train_list = sdsvc_trial_list
test_list = {}
model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 10249, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

torch.backends.cudnn.benchmark = False
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device("cuda:0")

# train_data = CSVDataSet(train_list)
# train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=True)
train_data = PickleDataSet(train_list)
train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
batch_sampler=None, num_workers=8, collate_fn=None,\
pin_memory=False, drop_last=False, timeout=0,\
worker_init_fn=None, multiprocessing_context=None)

model = model_bank.get_model(model_id, model_metric, model_settings, _)
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint['model'], strict=True)
model = model.to(device)
model.eval()

for count, (batch_x, batch_y) in enumerate(train_dataloader):
    batch_x = batch_x.to(device)
    label = batch_y[0]

    batch_y = torch.tensor([0]).to(device)
    
    with torch.no_grad():
        _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

    emb = emb.squeeze().data.cpu().numpy()
    
    if label not in test_list.keys():
        test_list[label] = emb[None, :]
    else:
        print('repeat eer:', label)
        break
#         test_list[label] = np.append(test_list[label], emb[None, :], axis=0)
    
    if (count+1) % 500 == 0:
        print(count+1)

del model, batch_x, batch_y
torch.cuda.empty_cache()

Using model bank
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000


In [26]:
count+1

14490

In [27]:
len(test_list)

14490

In [28]:
import pickle
out = component_dir+'/'+AUX_DATA_NAME_3
with open(out, 'wb') as handle:
    pickle.dump(test_list, handle)

# model 4

In [29]:
AUX_DATA_NAME_4 = 'AUX_DATA_4'

In [30]:
model_path = '/Lun2/rzz/kaldi-master/egs/zhiyong/sre19/exp/Xvector_1L_fullevad/ckpt/min_eer.model'
model_id = 'Xvector_SAP_1L'
model_metric = 'AM_normfree_softmax_anneal_ce_head'

In [31]:
# train_list = '/Lun0/zhiyong/dataset/vox1_kaldi_test/vox1_kaldi_test.csv'
train_list = sdsvc_trial_list
test_list = {}
model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 10249, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

torch.backends.cudnn.benchmark = False
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device("cuda:0")

# train_data = CSVDataSet(train_list)
# train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=True)
train_data = PickleDataSet(train_list)
train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
batch_sampler=None, num_workers=8, collate_fn=None,\
pin_memory=False, drop_last=False, timeout=0,\
worker_init_fn=None, multiprocessing_context=None)

model = model_bank.get_model(model_id, model_metric, model_settings, _)
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint['model'], strict=True)
model = model.to(device)
model.eval()

for count, (batch_x, batch_y) in enumerate(train_dataloader):
    batch_x = batch_x.to(device)
    label = batch_y[0]

    batch_y = torch.tensor([0]).to(device)
    
    with torch.no_grad():
        _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

    emb = emb.squeeze().data.cpu().numpy()
    
    if label not in test_list.keys():
        test_list[label] = emb[None, :]
    else:
        print('repeat eer:', label)
        break
#         test_list[label] = np.append(test_list[label], emb[None, :], axis=0)
    
    if (count+1) % 500 == 0:
        print(count+1)

del model, batch_x, batch_y
torch.cuda.empty_cache()

Using model bank
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000


In [32]:
count+1

14490

In [33]:
len(test_list)

14490

In [34]:
import pickle
out = component_dir+'/'+AUX_DATA_NAME_4
with open(out, 'wb') as handle:
    pickle.dump(test_list, handle)

# Ori score

In [55]:
score_1 = '/Lun2/rzz/kaldi-master/egs/zhiyong/sdsvc_exp/backend_exp/Standard_ETDNN_plus_c/score_cosine'
score_2 = '/Lun2/rzz/kaldi-master/egs/zhiyong/sre19/sdsvc_fullmix_etdnn/score_cosine_370'
score_3 = '/Lun2/rzz/kaldi-master/egs/zhiyong/sre19/sdsvc_fullmix_sparse/score_cosine_370'
score_4 = '/Lun2/rzz/kaldi-master/egs/zhiyong/sre19/sdsvc_fullmix_end/score_cosine_370'

# Aux score

In [ ]:
# model 1

In [35]:
SCORING_AUX_COSINE_1 = 'score_aux_cosine_1'
trail_path = sdsvc_trial_keys
score_out_path = component_dir+'/'+SCORING_AUX_COSINE_1

In [36]:
import pickle
with open(component_dir+'/'+AUX_DATA_NAME_1, 'rb') as handle:
    test_list = pickle.load(handle)

In [37]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [38]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [ ]:
# model 2

In [39]:
SCORING_AUX_COSINE_2 = 'score_aux_cosine_2'
trail_path = sdsvc_trial_keys
score_out_path = component_dir+'/'+SCORING_AUX_COSINE_2

In [40]:
import pickle
with open(component_dir+'/'+AUX_DATA_NAME_2, 'rb') as handle:
    test_list = pickle.load(handle)

In [41]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [42]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [ ]:
# model 3

In [43]:
SCORING_AUX_COSINE_3 = 'score_aux_cosine_3'
trail_path = sdsvc_trial_keys
score_out_path = component_dir+'/'+SCORING_AUX_COSINE_3

In [44]:
import pickle
with open(component_dir+'/'+AUX_DATA_NAME_3, 'rb') as handle:
    test_list = pickle.load(handle)

In [45]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [46]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


In [ ]:
# model 4

In [47]:
SCORING_AUX_COSINE_4 = 'score_aux_cosine_4'
trail_path = sdsvc_trial_keys
score_out_path = component_dir+'/'+SCORING_AUX_COSINE_4

In [48]:
import pickle
with open(component_dir+'/'+AUX_DATA_NAME_4, 'rb') as handle:
    test_list = pickle.load(handle)

In [49]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [50]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

trn_172338.wav trn_172362.wav target

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000


# fusion

In [65]:
from score import *
import score.apply_calibration as apply_calibration
import importlib
importlib.reload(apply_calibration)

<module 'score.apply_calibration' from '../score/apply_calibration.py'>

In [53]:
aux_score_file_1 = component_dir+'/'+ SCORING_AUX_COSINE_1
aux_score_file_2 = component_dir+'/'+ SCORING_AUX_COSINE_2
aux_score_file_3 = component_dir+'/'+ SCORING_AUX_COSINE_3
aux_score_file_4 = component_dir+'/'+ SCORING_AUX_COSINE_4

In [67]:
aux_key_file = sdsvc_trial_keys
calib_score_file = component_dir+'/'+'fusion'
linear_model_pth = component_dir+'/'+'calib.pth'

# calibrating(linear_model_pth, 50, aux_key_file, [aux_score_file_1, aux_score_file_2, aux_score_file_3, aux_score_file_4])
apply_calibration.applying_tmp(linear_model_pth, [score_1, score_2, score_3, score_4], calib_score_file)